In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from konlpy.tag import Kkma
tagger= Kkma()

## 1. RNNCELL

In [2]:
input_size = 10  # input dimension (word embedding) 0
hidden_size = 30 # Hidden dimension H
batch_size = 3   
length = 4       # T

In [3]:
rnncell = nn.RNNCell(input_size = input_size, hidden_size = hidden_size, bias = True, nonlinearity = 'tanh')

In [4]:
rnncell 

RNNCell(10, 30)

In [10]:
input = Variable(torch.randn(length, batch_size, input_size)) # T, B, D (embedding matrix에서 인덱싱한 워드벡터)

In [11]:
input.size()

torch.Size([4, 3, 10])

In [13]:
input[0].size()

torch.Size([3, 10])

In [5]:
input = Variable(torch.randn(length, batch_size, input_size)) # T, B, D (embedding matrix에서 인덱싱한 워드벡터)
hidden = Variable(torch.zeros(batch_size, hidden_size))       # first hidden state 첫 히든 스테이트 0으로 초기화
output = []
for i in range(length):
    hidden = rnncell(input[i], hidden)
    output.append(hidden)

In [6]:
hidden.size()

torch.Size([3, 30])

In [7]:
len(output)

4

In [8]:
input = Variable(torch.randn(length, batch_size, input_size)) # T, B, D (embedding matrix에서 인덱싱한 워드벡터)
hidden = Variable(torch.zeros(batch_size, hidden_size))       # first hidden state 첫 히든 스테이트 0으로 초기화
output = []
hidden = rnncell(input, hidden)

RuntimeError: input has inconsistent input_size: got 3, expected 10

## 2.RNN

In [14]:
input_size = 10
hidden_size = 30
batch_size = 3
length = 4
output_size = 5

In [15]:
rnn = nn.RNN(input_size, hidden_size, batch_first = True) 
# num_layers = 1, bias = True, nonlinearity = 'tanh', batch_first = True, dropout = 0, bidirectional = False

input = Variable(torch.randn(batch_size, length, input_size)) # B, T, D <= batch_first
hidden = Variable(torch.zeros(1,batch_size, hidden_size))     # 1,B,H   <= (num_layers * num_directions, batch, hidden_size)
output, hidden = rnn(input, hidden)

In [17]:
output.size()

torch.Size([3, 4, 30])

In [18]:
hidden.size()

torch.Size([1, 3, 30])

In [10]:
output.size() # output의 size = (3,4,30)

torch.Size([3, 4, 30])

In [11]:
hidden.size() # hidden의 size = (1, 3,30)

torch.Size([1, 3, 30])

In [12]:
linear = nn.Linear(hidden_size, output_size)
output = F.softmax(linear(output),1)
output.size() # 30 -> 5

torch.Size([3, 4, 5])

## 3. Bidirectional RNN

In [13]:
rnn = nn.RNN(input_size, hidden_size, num_layers=1, bias = True, nonlinearity = 'tanh', batch_first = True, dropout = 0, bidirectional = True)

In [14]:
input = Variable(torch.randn(batch_size, length, input_size)) # B, T, D
hidden = Variable(torch.zeros(2,batch_size,hidden_size))      # 2. B. H
                                                              # (num_layers * num_directions, batch_size, hiddem_size)

In [15]:
output, hidden = rnn(input, hidden)

In [16]:
output.size() # 3,4,60

torch.Size([3, 4, 60])

In [17]:
hidden.size() # 2,3,30 이게 양방향으로 쌓여서 output에서는 30 이 60으로...

torch.Size([2, 3, 30])

## 4. Multi-layer RNN


In [20]:
rnn = nn.RNN(input_size, hidden_size, num_layers=3, bias = True, nonlinearity = 'tanh', batch_first = True, dropout = 0,bidirectional = True )
input = Variable(torch.randn(batch_size, length, input_size)) # B, T, D
hidden = Variable(torch.zeros(3*2,batch_size , hidden_size))  # 6, B, H 
# (num_layers * num_directions, batch, hidden_size)

In [22]:
output, hidden = rnn(input,hidden)

In [25]:
output.size() # 3, 4, 60

torch.Size([3, 4, 60])

In [26]:
hidden.size() # 6, 3, 30

torch.Size([6, 3, 30])

In [19]:
size_2 = torch.randn(1,3)

In [25]:
size_2.size()

torch.Size([1, 3])

In [20]:
size_3 = torch.randn(1,3,4)

In [21]:
torch.bmm(size_2, size_3)

RuntimeError: Expected 3-dimensional tensor, but got 2-dimensional tensor for argument #1 'batch1' (while checking arguments for bmm)

## GRU

In [28]:
rnn = nn.GRU(input_size, hidden_size,batch_first = True) #,num_layers=1,bias=True,batch_first=True,bidirectional=True)

In [31]:
input = Variable(torch.randn(batch_size,length, input_size)) # B, T, D
hidden = Variable(torch.zeros(1, batch_size,hidden_size)) # 2, B, H  (bidirectional = True)

In [32]:
output, hidden = rnn(input, hidden)

In [34]:
print(output.size())

torch.Size([3, 4, 30])


In [35]:
print(hidden.size())

torch.Size([1, 3, 30])
